in the name of $ALLAH$

In [1]:
from sage.sat.boolean_polynomials import solve as solve_sat
from sage.rings.polynomial.multi_polynomial_sequence import PolynomialSequence
import re
import pdb

In [2]:
# It’s possible to compile code in a notebook cell with Cython. For this you need to load the Cython magic:
# Then you can define a Cython cell by writing %%cython on top of it. Like this:
# %%cython
# cython codes
%load_ext cython

In [9]:
%%cython -I ./

from libc.stdint cimport uint32_t, uint64_t
from libc.stdlib cimport malloc, calloc, free
    
cdef extern from "keeloq.c":
    void keeloq_encrypt(uint64_t *key, uint32_t *plaintext, uint32_t *ciphertext, int nrounds)
    void keeloq_decrypt(uint64_t *key, uint32_t *plaintext, uint32_t *ciphertext, int nrounds)
    
cdef extern from "polygen.c":
    ctypedef struct polynomial:
        char poly[300]
    uint64_t calculate_num_of_equations(int r, int number_of_plains)
    void polynomials(uint32_t *plains, uint32_t *ciphers, polynomial *equations, int r, int number_of_plains)

def keeloq_enc(k, p, r):
    cdef uint64_t k1
    cdef uint32_t p1
    cdef uint32_t c1
    k1, p1 = k, p
    keeloq_encrypt(&k1, &p1, &c1, r)
    return c1

def keeloq_dec(k, c, r):
    cdef:
        uint64_t k1
        uint32_t p1
        uint32_t c1
    k1, c1 = k, c
    keeloq_decrypt(&k1, &p1, &c1, r)
    return p1
    
def keeloq_polys(ptexts, ctexts, r):
    cdef:
        int number_of_plaintexts = len(ptexts)
        uint64_t neqs        
        uint32_t *plains = <uint32_t *> calloc(number_of_plaintexts, sizeof(uint32_t))
        uint32_t *ciphers = <uint32_t *> calloc(number_of_plaintexts, sizeof(uint32_t))
        polynomial *equations
        
    for i in range(number_of_plaintexts):
        plains[i] = ptexts[i]
        ciphers[i] = ctexts[i]
    neqs = calculate_num_of_equations(r, number_of_plaintexts)
    equations = <polynomial *>malloc(neqs * sizeof(polynomial))
    polynomials(plains, ciphers, equations, r, number_of_plaintexts)
    free(plains)
    free(ciphers)
    st = []
    for i in range(neqs):
        st.append(equations[i].poly)
    free(equations)
    return st

In [10]:
k = 0x5cec6701b79fd949
p = 0xf741e2db
r = 528
c = keeloq_enc(k, p, r)
hex(c)

'0xe44f4cdf'

In [12]:
@parallel
def paralle_encrypt(k, p, r):
    return keeloq_enc(k, p, r)

In [13]:
def serial_encrypt(k, p, r):
    ciphers = []
    for p in plains:
        ciphers.append(keeloq_enc(k, p, r))
        
    return ciphers

In [14]:
r = 528
k = 0x5cec6701b79fd949
number_of_plains = 2^3
plains = [randint(0, 2^32 - 1) for _ in range(number_of_plains)]
inputs = [(k, plains[i], r) for i in range(len(plains))]

In [17]:
%time ciphers = paralle_encrypt(inputs)

CPU times: user 53 µs, sys: 7 µs, total: 60 µs
Wall time: 57 µs


In [18]:
%time serial_encrypt(k, plains, r)

CPU times: user 53 µs, sys: 8 µs, total: 61 µs
Wall time: 62.9 µs


[3965727524,
 2232568784,
 2842232833,
 1742918013,
 2437120505,
 288824108,
 795992873,
 779108897]

In [19]:
k = 0x5cec6701b79fd949
r = 128
number_of_plains = 2
plains = [randint(0, 2^32 - 1) for _ in range(number_of_plains)]
ciphers = [0]*number_of_plains
for i in range(number_of_plains):
    ciphers[i] = keeloq_enc(k, plains[i], r)
%time temp = keeloq_polys(plains, ciphers, r)
vrs = []
for f in temp:
    terms = re.split(' \+ |\*', f)
    for v in terms:
        if (not v.isdigit()):
            vrs.append(v)
vrs = list(set(vrs))
R = BooleanPolynomialRing(len(vrs), vrs)
%time ps = map(R, temp)
ps = PolynomialSequence(ps)

CPU times: user 3.45 ms, sys: 0 ns, total: 3.45 ms
Wall time: 3.45 ms
CPU times: user 10 s, sys: 0 ns, total: 10 s
Wall time: 10 s


In [20]:
ps

Polynomial Sequence with 896 Polynomials in 896 Variables

In [ ]:
%time sls = solve_sat(ps, n = infinity, s_verbosity = 4)
#print(sls)
print("number of solutions : %d" % len(sls))

In [349]:
def guss_keys(ps, ng, original_key):
    original_key = bin(original_key)[2:].zfill(64)
    original_key = list(original_key)
    original_key.reverse()
    original_key = map(R, original_key)
    gps = []
    for p in ps:
        gps.append(p)
    for i in range(ng):
        temp = "k_%d + %d" % (i, original_key[i])
        temp = R(temp)
        gps.append(temp)
    return PolynomialSequence(gps)

In [347]:
k = 0x5cec6701b79fd949
gps = guss_keys(ps, 30, k)
print(ps)
print(gps)

Polynomial Sequence with 896 Polynomials in 896 Variables
Polynomial Sequence with 926 Polynomials in 896 Variables


In [348]:
%time sls = solve_sat(gps, n = infinity, s_verbosity = 4)
#print(sls)
print("number of solutions : %d" % len(sls))

CPU times: user 8min 42s, sys: 128 ms, total: 8min 42s
Wall time: 8min 42s
number of solutions : 1


In [303]:
def check_correctness(sls, original_key):
    original_key = bin(original_key)[2:].zfill(64)
    original_key = list(original_key)
    original_key.reverse()
    for n in range(0, len(sls)):
        matching_flag = True
        key_sol = dict()
        kindex = []
        for i in range(64):
            kname = 'k_%d' % i
            if kname in R.variable_names():
                kname = R(kname)
                kval = sls[n].get(kname)
                if kval != None:
                    kindex.append(i)
                    key_sol[kname] = kval        
        for i in kindex:
            if original_key[i] != str(key_sol[R("k_%d" % i)]):
                matching_flag = False
                break
        #pdb.set_trace()
        if matching_flag == True:
            print("C0ngratulati0n! sls[%d] matches with the original key :)" % n)
            print("Number of recoverd key bits: %d" % len(list(key_sol)))
            print(key_sol)
            return
    print("There is no solution matching with the original key :(")

In [304]:
check_correctness(sls, k)

C0ngratulati0n! sls[0] matches with the original key :)
Number of recoverd key bits: 64
{k_39: 0, k_31: 1, k_9: 0, k_57: 0, k_0: 1, k_35: 0, k_33: 0, k_6: 1, k_53: 1, k_37: 0, k_2: 0, k_19: 1, k_4: 0, k_10: 0, k_12: 1, k_14: 1, k_16: 1, k_25: 1, k_47: 0, k_29: 1, k_49: 0, k_43: 0, k_21: 0, k_45: 1, k_27: 0, k_63: 0, k_59: 1, k_61: 0, k_58: 1, k_56: 0, k_23: 1, k_41: 1, k_15: 1, k_51: 1, k_32: 1, k_30: 0, k_36: 0, k_1: 0, k_34: 0, k_8: 1, k_7: 0, k_52: 0, k_3: 1, k_38: 0, k_5: 0, k_18: 1, k_13: 0, k_17: 1, k_11: 1, k_46: 1, k_26: 1, k_48: 0, k_24: 1, k_22: 0, k_42: 1, k_20: 1, k_44: 0, k_60: 1, k_28: 1, k_62: 1, k_50: 1, k_54: 1, k_40: 1, k_55: 1}
